In [32]:
import pickle
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import svm
import random
import math
import os

In [33]:
context_feat = pickle.load(open("pickles/alf_context_feat.pkl", "rb"))
location_feat = pickle.load(open("pickles/alf_location.pkl", "rb"))
pos_feat = pickle.load(open("pickles/alf_pos_feat.pkl", "rb"))
year_diff = pickle.load(open("pickles/year_diff.pkl", "rb"))
sync = pickle.load(open("pickles/en_synctatic_feat.pkl", "rb"))

In [34]:
a1 = pickle.load(open("pickles/alf_sim_con_abs_data.pkl", "rb"))
a2 = pickle.load(open("pickles/alf_sim_con_intro_data.pkl", "rb"))
a3 = pickle.load(open("pickles/alf_sim_con_conc_data.pkl", "rb"))
a4 = pickle.load(open("pickles/alf_sim_con_rest_data.pkl", "rb"))
a5 = pickle.load(open("pickles/alf_sim_title_abs_data.pkl", "rb"))
a6 = pickle.load(open("pickles/alf_sim_title_intro_data.pkl", "rb"))
a7 = pickle.load(open("pickles/alf_sim_title_conc_data.pkl", "rb"))
a8 = pickle.load(open("pickles/alf_sim_title_rest_data.pkl", "rb"))

In [35]:
tags = pickle.load(open("../pickles_data/baseline_tags.pkl", "rb"))

In [36]:
dataset = []
output = []
for key in tags :
    if key in a1 :
        papers = tags[key]
        for pap1 in papers :
            paper = pap1['paper_name']
            data = []

            for pap in context_feat[key]:
                if(pap['paper_name']==paper) :
                    data.extend(pap['context_feat'])
                    break

            for pap in location_feat[key]:
                if(pap['paper_name']==paper) :
                    data.append(pap['whole'])
                    data.append(pap['intro'])
                    data.append(pap['relwork'])
                    data.append(pap['rest'])
                    break

            for pap in pos_feat[key] :
                if(pap['paper_name']==paper) :
                    data.extend(pap['position_feat'])
                    break


            for pap in year_diff[key] :
                if(pap['paper_name']==paper) :
                    data.append(pap['diff'])
                    break

            for pap in a2[key] :
                if(pap['paper_name']==paper) :
                    for key1 in pap :
                        if(key1!='paper_name') :
                            data.append(pap[key1])
                    break

            for pap in a3[key] :
                if(pap['paper_name']==paper) :
                    for key1 in pap :
                        if(key1!='paper_name') :
                            data.append(pap[key1])
                    break

            for pap in a4[key] :
                if(pap['paper_name']==paper) :
                    for key1 in pap :
                        if(key1!='paper_name') :
                            data.append(pap[key1])
                    break


            for pap in a5[key] :
                if(pap['paper_name']==paper) :
                    for key1 in pap :
                        if(key1!='paper_name') :
                            data.append(pap[key1])
                    break



            for pap in a6[key] :
                if(pap['paper_name']==paper) :
                    for key1 in pap :
                        if(key1!='paper_name') :
                            data.append(pap[key1])
                    break


            for pap in a7[key] :
                if(pap['paper_name']==paper) :
                    for key1 in pap :
                        if(key1!='paper_name') :
                            data.append(pap[key1])
                    break


            for pap in a8[key] :
                if(pap['paper_name']==paper) :
                    for key1 in pap :
                        if(key1!='paper_name') :
                            data.append(pap[key1])
                    break

            for pap in a1[key] :
                if(pap['paper_name']==paper) :
                    for key1 in pap :
                        if(key1!='paper_name') :
                            data.append(pap[key1])
                    break
                    
            for pap in sync[key] :
                if(pap['paper_name']==paper) :
                    bool_list = pap['pos_feat']
                    feat_list = []
                    for val in bool_list :
                        if(val):
                            feat_list.append(1)
                        else :
                            feat_list.append(0)
                    data.extend(feat_list)
                    break
                    
            if(len(data)==30) :
                dataset.append(data)  
                output.append(pap1['tag'])

In [37]:
print(len(dataset))
print(len(output))

27600
27600


In [48]:
data, out = shuffle(dataset, output)

In [51]:
print(len(out))
print(len(data))

5474
5474


In [52]:
values = np.array(data)
output = np.array(out)

In [53]:
print(values.shape)
print(output.shape)

(5474, 30)
(5474,)


In [54]:
scaler = MinMaxScaler((0,1))
values = scaler.fit_transform(values)

In [55]:
def split(data, output):
    n = len(data)
    last = int(0.8*n)
    train_data = data[:last]
    train_output = output[:last]
    test_data = data[last:]
    test_output = output[last:]
    return train_data, test_data, train_output, test_output

In [56]:
train_data, test_data, train_output, test_output = split(values, output)

In [57]:
def shuffle(train_data, train_output) :
    baselines = []
    non_baselines = []
    for i in range(len(train_output)) :
        if(train_output[i]==1) :
            baselines.append(train_data[i])
        else :
            non_baselines.append(train_data[i])
    
    n = len(baselines)
    ar = np.random.choice(len(non_baselines), len(baselines))
    nb_ar = []
    for x in ar :
        nb_ar.append(non_baselines[x])
        
    data = []
    data.extend(nb_ar)
    output = []
    for i in range(len(data)):
        output.append(0)
    
    data.extend(baselines)
    for i in range(n) :
        output.append(1)
        
    total_data = []
    for i in range(len(data)):
        ar = []
        ar.append(data[i])
        ar.append(output[i])
        total_data.append(ar)
        
    total_data = np.array(total_data)
    np.random.shuffle(total_data)
    
    data = []
    output = []
    for ar in total_data :
        data.append(ar[0])
        output.append(ar[1])
        
    data = np.array(data)
    output = np.array(output)
    
    return data, output

In [31]:
# train_data, train_output = shuffle(train_data, train_output)

In [58]:
params = {'C':[0.1, 0.5, 1, 2]}
modelin = svm.SVC(kernel='rbf', max_iter=1e4, gamma='scale')
model = GridSearchCV(modelin, params, cv=2, n_jobs=5)
# model = LogisticRegression(solver='lbfgs',max_iter=1e4, n_jobs=5, random_state=1, warm_start=False, C=1, fit_intercept=True)
clf = model.fit(train_data, train_output)
predict_test = clf.predict(test_data)
print(classification_report(test_output, predict_test))
predictions = clf.predict(train_data)
print(classification_report(train_output, predictions))
print(model.best_params_)
# print(type(model.best_estimator_))

              precision    recall  f1-score   support

           0       0.82      0.81      0.82       568
           1       0.80      0.81      0.81       527

    accuracy                           0.81      1095
   macro avg       0.81      0.81      0.81      1095
weighted avg       0.81      0.81      0.81      1095

              precision    recall  f1-score   support

           0       0.81      0.81      0.81      2169
           1       0.82      0.82      0.82      2210

    accuracy                           0.81      4379
   macro avg       0.81      0.81      0.81      4379
weighted avg       0.81      0.81      0.81      4379

{'C': 2}


In [17]:
print(values[1])

[1.         1.         1.         0.         0.         0.
 0.02857143 0.125      0.         0.         1.         0.
 0.02622169 0.         0.         0.74040879 0.27238365 0.2394133
 0.         0.05875921 0.         0.04543682 0.35801768 0.
 0.         0.         0.         1.         0.         0.        ]


In [18]:
#GraLap
W = np.zeros((len(values),len(values)))

In [163]:
td = list(train_data)
testd = list(test_data)
outd = list(train_output)
outtd = list(test_output)
td.extend(testd)
outd.extend(outtd)
print(len(td))
print(len(outd))
data = np.array(td)
out = np.array(outd)
print(data.shape)
print(out.shape)

9940
9940
(9940, 30)
(9940,)


In [164]:
power_val = np.power(data, 30)

In [165]:
a =  power_val[np.newaxis,:,:]
b = power_val[:,np.newaxis,:]

In [166]:
# count = 0
# power_val = np.power(values,len(values[0]))
# diff = power_val[np.newaxis,:,:]-power_val[:,np.newaxis,:]
# print(diff.shape)
diff = a-b
print(diff.shape)
# for i in range(len(values)) :
#     for j in range(len(values)) :
#         sum1 = 0
#         vi = power_val[i]
#         vj = power_val[j]
#         v = vi-vj
#         v = np.sum(v)
#         v = -1*v
#         W[i][j] = np.exp(v)
#     count+=1
#     if(count%100==0) :
#         print(count)

(9940, 9940, 30)


In [167]:
diff = np.sum(diff, axis=2)
print(diff.shape)

(9940, 9940)


In [168]:
diff = -1*diff
diff = diff/9
W = np.exp(diff)


In [169]:
print(W.shape)

(9940, 9940)


In [170]:
T = np.zeros((len(values), len(values)))

In [171]:
T = W/np.sum(W,axis=0, keepdims=True)

In [172]:
print(T[0][0])

7.912564266225619e-05


In [173]:
print(W[0][0]/np.sum(W[:,0]))

7.912564266225701e-05


In [174]:
outlb = []
for i in range(len(out)) :
    if(i<len(train_data)) :
        if(out[i]==0) :
            outlb.append([1,0])
        else :
            outlb.append([0,1])
    else :
        outlb.append([0,0])

In [175]:
outlb = np.array(outlb)
print(outlb.shape)

(9940, 2)


In [176]:
for i in range(100) :
    outlb = np.matmul(T, outlb)
    outlb = outlb/np.sum(outlb, axis=1, keepdims=True)
    for j in range(len(train_data)) :
        if(out[j]==0) :
            outlb[j][0]=1
            outlb[j][1]=0
        else :
            outlb[j][0]=0
            outlb[j][1]=1     

In [177]:
test_out = outlb[len(train_data):]
print(test_out.shape)

(5520, 2)


In [178]:
test = []
for i in range(len(test_out)):
    if(test_out[i][0]>test_out[i][1]) :
        test.append(0)
    else :
        test.append(1)

In [179]:
print(sum(test))

5077


In [180]:
print(classification_report(test_output, np.array(test)))

              precision    recall  f1-score   support

           0       0.91      0.08      0.15      4993
           1       0.10      0.92      0.17       527

    accuracy                           0.16      5520
   macro avg       0.50      0.50      0.16      5520
weighted avg       0.83      0.16      0.15      5520



In [181]:
print(test_out)

[[0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 ...
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]]
